In [ ]:
import gc
import torch
import torchvision
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
from io import BytesIO

In [ ]:
# a) Tải mô hình Faster R-CNN pre-trained trên COCO
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=weights)
model.eval()

# Danh sách nhãn của COCO
COCO_CLASSES = weights.meta["categories"]



In [ ]:
def predict_and_show(url, threshold=0.8):
    # Tải ảnh từ URL
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    
    # Tiền xử lý
    transform = T.Compose([T.ToTensor()])
    img_tensor = transform(img)

    # Dự đoán
    with torch.no_grad():
        prediction = model([img_tensor])

    # Hiển thị
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img)

    for i in range(len(prediction[0]['scores'])):
        score = prediction[0]['scores'][i].item()
        if score > threshold:
            box = prediction[0]['boxes'][i].detach().cpu().numpy()
            label_id = prediction[0]['labels'][i].item()
            label_name = COCO_CLASSES[label_id]

            rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], 
                                     linewidth=2, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)
            plt.text(box[0], box[1], f'{label_name}: {score:.2f}', 
                     color='black', bbox={'facecolor': 'lime', 'alpha': 0.5})

    plt.axis('off')
    plt.title("Kết quả nhận diện bằng Faster R-CNN (Dữ liệu Online)")
    plt.show()

In [ ]:
url_1 = "http://images.cocodataset.org/val2017/000000039769.jpg" # Ảnh hai con mèo
# url_2 = "http://images.cocodataset.org/val2017/000000000139.jpg" # Ảnh phòng khách
# url_3 = "http://images.cocodataset.org/val2017/000000000632.jpg" # Ảnh đường phố

# Chạy thử
predict_and_show(url_1)
     